In [12]:
import os, re
import pandas as pd
import pickle
from collections import Counter

# Create aggregated text files containg userinfo and hashtags

## 1. make sure there are no duplicates in source material

In [23]:
clean_totalUsertexts_path = "/Volumes/mos_storage/ma/data/clean_totalUsertexts/"

In [20]:
path1 = "/Volumes/mos_storage/ma/data/totalUsertexts/"

In [25]:
files1 = []
for root, dirs, files in os.walk(path1):
    for file in files:
        if file.endswith(".txt"):
            files1.append(file)

for index, file in enumerate(files1):
    with open(path1 + file) as f:
        lines = f.readlines()
        lines_set = set(lines)
        outfile = open(clean_totalUsertexts_path + file,"w")
        for line in lines_set:
            outfile.write(line)
        outfile.close()
    print("written clean file: " + '{:<30}'.format(file) + " number " + str(index),end="\r")

In [26]:
# number of files processed in path1
print(str(len(files1)))

482151


In [27]:
path2 = "/Volumes/mos_storage/merged_totalUsertexts/"

In [28]:
files2 = []
for root, dirs, files in os.walk(path2):
    for file in files:
        if file.endswith(".txt"):
            files2.append(file)

for index, file in enumerate(files1):
    with open(path2 + file) as f:
        lines = f.readlines()
        lines_set = set(lines)
        outfile = open(clean_totalUsertexts_path + file,"a") #NOTE! We append to an existing file from above!
        for line in lines_set:
            outfile.write(line)
        outfile.close()
    print("written clean file: " + '{:<30}'.format(file) + "no " + str(index),end="\r")

In [29]:
# number of files processed in path2
print(str(len(files2)))

482151


## How many tweets (approximately) do we have in the cleaned dataset?

In [31]:
clean_files = []
for root, dirs, clean_totalUsertexts_pathtotalUsertexts_path in os.walk(clean_totalUsertexts_path):
    for file in files:
        if file.endswith(".txt"):
            clean_files.append(file)

counter = 0
for index, file in enumerate(clean_files):
    with open(clean_totalUsertexts_path + file) as f:
        lines = f.readlines()
        counter += len(lines)
        print("Finnished counting lines in file: " + '{:<25}'.format(file) + " no " + str(index),end="\r")

In [32]:
print("Approximate number of tweets in dataset is " + str(counter))

Approximate number of tweets in dataset is 334510191


## 2. Parse out each #hashtag in each aggregated userfile
Parse out every tweet from each user that contains one or more hashtags

In [36]:
!find /Volumes/mos_storage/ma/data/clean_totalUsertexts/ -maxdepth 1 -name '*.txt' -exec egrep -H '#\w+' '{}' \; > /Volumes/mos_storage/ma/data/medieforsk/grepresult.txt

###2.1 Parse out the user and all hashtags for each line in the resultfile 

In [ ]:
hashtag_pattern = re.compile(r'#\w+')
username_pattern = re.compile(r'//(\w+)')

df = pd.DataFrame()
#with open("/Volumes/mos_storage/ma/data/medieforsk/grepresult.txt") as f:
with open("/Volumes/mos_storage/ma/data/medieforsk/greptestest") as f:
 username line in f:
        user_dict = {}
        
        username_match = username_patter.search(line)
        username = username_match.groups(1)
        user_dict["name"] = username
        
        hastag_matches = hashtag_pattern.findall(line)
        user_dict["hashtags"] = hashtag_matches
        
        df.append(user_dict)

In [5]:
def parse_users_and_hashtags_from_grep_output(grep_file):
    """ Takes a textfile with grep-results as lines, parses out the 
    usernames and hashtags and returns a Pandas DataFrame.
    """
    hashtag_pattern = re.compile(r'#\w+')
    username_pattern = re.compile(r'//(\w+)')

    ######## Data structure
    ## {"edvinsson":{"hashtags":["#svpol","#mat",...]

    #with open("/Volumes/mos_storage/ma/data/medieforsk/grepresult.txt") as f:

    with open(grep_file) as f:
        user_dict = {}
        for line in f:
        
            username_match = username_pattern.search(line)
            username = username_match.groups(1)[0] # returns tuple, get the string value by indexing
        
            hashtag_matches = hashtag_pattern.findall(line)
        
            if username in user_dict.keys():
                current_hashtags = user_dict[username]["hashtags"]
                current_hashtags.extend(hashtag_matches)
                user_dict[username]["hashtags"] = current_hashtags
        
            else:
                user_dict[username] = {}
                user_dict[username]["hashtags"] = hashtag_matches
    
    df = pd.DataFrame(user_dict)
    return df.T # transpose the dataFrame som that user names are the indices and hashtags is a column

In [6]:
result = parse_users_and_hashtags_from_grep_output("/Volumes/mos_storage/ma/data/medieforsk/grepresult.txt")

In [7]:
pickle.dump(result,open("/Volumes/mos_storage/ma/data/medieforsk/result.pickle","wb"))

In [ ]:
for tag in result.ix["mattiasostmar"]:
    print(tag)

In [17]:
tag_counter = Counter()

In [24]:
for list in result["hashtags"]:
    tag_counter.update(list)

# How many hashtags do we have in the data set?

In [29]:
len(tag_counter.keys())

4723964

# What are the most common hashtags in the data set?

In [31]:
tag_counter.most_common(10)

[('#svpol', 1668308),
 ('#fb', 387621),
 ('#melfest', 358798),
 ('#RisingStar', 343018),
 ('#twittpuck', 336983),
 ('#NowPlaying', 281519),
 ('#TheFoooConspiracy', 251317),
 ('#KCA', 232099),
 ('#twittboll', 224622),
 ('#RunKeeper', 194446)]

## Most common hashtags per month

In [17]:
hashtag_pattern = re.compile(r'#\w+')

hashtags_dict = {}

##### we want this data structure: 

# {"1":["#abc","#def","#ghi"...],
# "2":["#abc","#zyx","...]}

for month in months:
    monthly_hashtags_list = []
    
    current_months_path = datapath + month + "/"
    current_months_files = []
    for root, dirs, files in os.walk(current_months_path):
        print("looking in: " + current_months_path)
        for file in files:
            if file.endswith(".txt"):
                current_months_files.append(file)
            else:
                continue
                
    
    for file in current_months_files:
        username = file[:-4]
        with open(current_months_path + file) as the_file:
            for line in the_file:
                hashtags = hashtag_pattern.findall(line)
                print(hashtags)
                for hashtag in hashtags:
                    monthly_hashtags_list.append(hashtag)
    
    hashtags_dict[month] = monthly_hashtags_list
    print("number of hashtags in month no " + month + ": " + str(len(hashtags_dict[month])))

looking in: /Volumes/backup_hd/ma/data/monthlyUsertexts/2015/1/
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#TheOC']
[]
['#allinikväll']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#behöverjuinteöverdriva']
[]
[]
['#intesåkaxigidag']
[]
[]
[]
[]
['#twittpuck', '#difse', '#fbkse', '#moritzgifs']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#piggsomaldrigförr']
['#FIFJ18']
['#mäktigt']
[]
['#gunvald']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#GoTSeason5']
['#rotaract']
['#DN']
[]
[]
[]
[]
[]
[]
[]
['#dressedtoimpress']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['#sjuottonåringenbiggie']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[